*Preprocessing*

Preprocessing, LDA end of day
must log into this terminal eltanin every time
!

*READ ME:*
*This file is to be used to preprocess data. How?* 
1) Reads data from mySQL and accesses original text data from file.
2) Processes and stores related ID and documentList connected to original text.
3) Updates original text, ID, and documentList attributes in mySQL database for usage in lda_pipeline

In [1]:
!pip install wheel
!pip install Cmake
!pip install gensim==3.8.3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [47]:
import os
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

/bin/bash: apt-get: command not found
openjdk version "1.8.0_322"
OpenJDK Runtime Environment (build 1.8.0_322-b06)
OpenJDK 64-Bit Server VM (build 25.322-b06, mixed mode)


Note:

This section pertains to reading csv files, and storing them as dataframes, this section is not needed to be run anymore but helpful for future reference. 

In [2]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' 

In [4]:
#Time to import from the CSV File, just kidding you need to import from mySql
import pandas as pd

df = pd.read_csv('tweets_USATODAY.csv')
df.head()


,author_id,conversation_id,created_at,entities,id,lang,public_metrics,text,retweet_count,like_count,reply_count,quote_count,qouted_id,replied_toid,urls
0,15754281,1586417588032053249,2022-10-29 18:00:01+00:00,"{'urls': [{'start': 228, 'end': 251, 'url': 'h...",1586417588032053249,en,"{'retweet_count': 4, 'reply_count': 12, 'like_...",A first-of-its-kind survey measures a potentia...,4,21,12,1,Null,Null,"[{'start': 228, 'end': 251, 'url': 'https://t...."
1,15754281,1585927131057356800,2022-10-28 09:31:08+00:00,"{'urls': [{'start': 222, 'end': 245, 'url': 'h...",1585927134106615810,en,"{'retweet_count': 3, 'reply_count': 3, 'like_c...","For the past two-plus years, trick-or-treaters...",3,9,3,0,Null,1585927131057356800,"[{'start': 222, 'end': 245, 'url': 'https://t...."
2,15754281,1585665123213000704,2022-10-27 16:10:00+00:00,"{'urls': [{'start': 253, 'end': 276, 'url': 'h...",1585665123213000704,en,"{'retweet_count': 15, 'reply_count': 11, 'like...",Carnival Cruise Line has further eased its COV...,15,27,11,3,Null,Null,"[{'start': 253, 'end': 276, 'url': 'https://t...."
3,15754281,1584844060535050242,2022-10-25 09:47:23+00:00,"{'urls': [{'start': 198, 'end': 221, 'url': 'h...",1584844060535050242,en,"{'retweet_count': 13, 'reply_count': 5, 'like_...",In Tuesday's paper:\n- Test scores plummet in ...,13,30,5,2,Null,Null,"[{'start': 198, 'end': 221, 'url': 'https://t...."
4,15754281,1582476616160141331,2022-10-18 21:00:00+00:00,"{'urls': [{'start': 192, 'end': 215, 'url': 'h...",1582476616160141331,en,"{'retweet_count': 17, 'reply_count': 6, 'like_...","BA.5 is losing its grip on the US, accounting ...",17,18,6,2,Null,Null,"[{'start': 192, 'end': 215, 'url': 'https://t...."


In [4]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [6]:
#Set csv file here aka uncomment which you currently want to run
myFile = "tweets_USATODAY.csv"
#csvFile = "tweets_nytimes.csv"
#csvFile = "tweets_WSJ.csv"

In [7]:
#Import original documents (plain text)
import csv
from tqdm import tqdm
completedocs = []
originaltext = []
with open(myFile, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in tqdm(csvreader):
        originaltext.append(row[7])

5931it [00:00, 43085.82it/s]


In [11]:
#Import and pre process
import csv
with open(myFile,'r') as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader)
            doclistNews, idlistNews = [], []
            for row in tqdm(csvreader):
                doclistNews.append(row[7])
                idlistNews.append(row[4])
            
            #call the pre-processing
            texts = preprocess_data(doclistNews, idlistNews, extra_stopwords = {},len_th=4,lemmatized=False)


5930it [00:00, 42262.11it/s]


In [54]:
#Check
#len(texts)

5859

MySQL Time

In [4]:
#Connect to da server
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

# To connect MySQL database
sql_db = MySQLdb.connect(host='127.0.0.1',user='covidAnalysis', password = "k34p63MbDDcZ9yf4",db='covid19framing')
cursor = sql_db.cursor()

#cursor.execute('desc twitter;')
#number_of_rows = cursor.execute("SELECT * FROM twitter")
#results = cursor.fetchall()

    # To close the connection




In [6]:
cursor.execute('show tables;')
#results = cursor.fetchall()


cursor.execute('desc articles;')
results = cursor.fetchall()

for col in results:
    print(col)


#Visual SQL find!

('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment')
('text', 'mediumtext', 'NO', '', None, '')
('date_published', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', '')
('url', 'mediumtext', 'NO', '', None, '')
('title', 'varchar(1000)', 'NO', '', None, '')
('source', 'varchar(60)', 'YES', '', None, '')
('language', 'varchar(2)', 'YES', '', None, '')
('date_scraped', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', '')
('state', 'varchar(14)', 'NO', '', None, '')
('text_preproc', 'mediumtext', 'YES', '', None, '')


In [7]:
print(results)
#print("Number of rows are:" ,number_of_rows)

(('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('text', 'mediumtext', 'NO', '', None, ''), ('date_published', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', ''), ('url', 'mediumtext', 'NO', '', None, ''), ('title', 'varchar(1000)', 'NO', '', None, ''), ('source', 'varchar(60)', 'YES', '', None, ''), ('language', 'varchar(2)', 'YES', '', None, ''), ('date_scraped', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', ''), ('state', 'varchar(14)', 'NO', '', None, ''), ('text_preproc', 'mediumtext', 'YES', '', None, ''))


In [8]:
cursor.execute('show tables;')
R = cursor.fetchall()

In [10]:
print(R)

(('article_author',), ('article_keyword',), ('article_organization',), ('articles',), ('organizations',), ('twitter',), ('twitter_hashtag',))


('article_keyword',)

In [9]:
#cursor = sql_db.cursor()
offset = 0#counter for offset
TEST = [0] #starting with a non-empty results to make sure we get into loop in the first place

while TEST:
    #cursor.execute("SELECT * FROM articles WHERE text_preproc IS NOT NULL;".format(offset))
    cursor.execute("select * from twitter where text_preproc IS NOT NULL limit 100 offset {0};".format(offset))
    #cursor.execute("select * from articles where text_preproc is not NULL;")
    res =  cursor.fetchall()
    #query = "SELECT * FROM articles WHERE text_preproc IS NOT NULL AND text_preproc NOT IN ({}) LIMIT 1000 OFFSET %s;".format(values_placeholder)
    offset += 100
    TEST = cursor.fetchall()
    if not TEST:  # Check if there are no more results
        break

In [13]:

res[6]
#print("Number of rows are:" ,number_of_rows)

(5532,
 None,
 '1322189888',
 '@TheAngryEpi @CDCgov I don’t know anyone with this option. As an “essential worker” we were abandoned by @CDCDirector and @WHCOVIDResponse and expected to repeatedly expose ourselves to positive #COVID19 cases with “tools” that didn’t protect us from infection and disability.',
 24,
 0,
 2,
 datetime.datetime(2022, 10, 31, 23, 50, 58),
 datetime.datetime(2023, 3, 7, 10, 58, 11),
 1587218683310403584,
 None,
 1587218683310403584,
 1587230681146527744,
 'theangryepi cdcgov anyone option essential worker abandoned cdcdirector whcovidresponse expected repeatedly expose positive covid cases tools protect infection disability')

In [6]:
#Stopword list
#read all the stop words and add them to the list of extra stop words..
extra_stop_words = open('Stopword_list.txt', 'r')
#addiong some stop words...
extra_stop_words_list = extra_stop_words.readlines()
extra_stopwords = set()
for item in extra_stop_words_list:
    extra_stopwords.add(item.strip())
extra_stopwords.add('amp');extra_stopwords.add('&amp');extra_stopwords.add('&amp;')

In [86]:
# Version control set up
# Get access to github repo and or branch.
#organization_id col to separate
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

cursorUpdate = sql_db.cursor()
offset = 0#counter for offset
results = [0] #starting with a non-empty results to make sure we get into loop in the first place
# Go through and fix each piece/evaluate 

while results:
    cursor.execute("select * from twitter where text_preproc is NULL limit 1000 offset {0};".format(offset))
    offset += 1000
    results = cursor.fetchall()
    if not results:  # Check if there are no more results
        break
    result_dic = {i[0]:i for i in results}

    #send to pre_process (...)
    out = preprocess_data([i[3] for i in results], [i[0] for i in results], extra_stopwords,len_th=4,lemmatized=False)
    #out ==> 1. pre_prcesed doc, original, id
    
    #putting it back to DB
    for o in out:
        cursor.execute("update twitter SET text_preproc = '{0}' where id = {1}".format(' '.join(o[0]),o[2]))
        print(o[0], o[2])
                
        
sql_db.commit()    

In [80]:
sql_db.commit()

Methods:

In [85]:
#pre-processing 
#this part uses Amin's code 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities

import csv, sys
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


import pandas as pd
import numpy as np

import copy


try:
	csv.field_size_limit(sys.maxsize)
except:
	print('Error in setting maxSize for CSV output')
    
def preprocess_data(doc_list, id_list, extra_stopwords = {},len_th=4,lemmatized=False):
	'''
	Returns: a list of process dataset and origianl documents of those documents

	This function removes stop-wrods, lemmatized the documens, if stated, and eliminates the documnets 
	with lenhgth of 4 or less. 
	***These processes may result in lower number of documents than the original number. To make sure 
	you receive both the original docs and the processed doc in similar order we return both.

	parameter doc_list: a list of string (documents)
	parameter extra_stopwords: NLTK.stop_words are used, if you wish to add to that list, you can use yours.
	parameter len_th: documents with len_th and less will be removed.
	parameter lemmatized: If true, the terms will be lemmatized. **be aware that lemmatization of the documents
	will result in different topics and may need different evaluation, including NPMI, stability, or human assessment**

	'''

	# replace single smart quote with single straight quote, so as to catch stopword contractions
	doc_list = [re.sub("[\u2018\u2019]", "'", doc) for doc in doc_list] #replace qoute with regualar qoutations
	doc_list = [re.sub('\d+', '', doc) for doc in doc_list]
	doc_list = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in doc_list]
	#doc_list = [re.sub('pdf|icon|jpg', '', doc) for doc in doc_list]
	#doc_list = [re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', doc) for doc in doc_list]
	doc_list = [re.sub(r"http\S+", '', doc) for doc in doc_list]

	# initialize regex tokenizer
	tokenizer = RegexpTokenizer(r'\w+')
	# create English stop words list
	en_stop = set(stopwords.words('english'))
	# add any extra stopwords
	if (len(extra_stopwords) > 0):
		en_stop = en_stop.union(extra_stopwords)

	#defining a lemmatizer
	lemmatizer = WordNetLemmatizer()

	# list for tokenized documents in loop
	texts = []
	original_docs = []
	# loop through document list
	c = 0 #counter on the document number
	for i in doc_list:
		# clean and tokenize document string
		raw = i.lower()
		tokens = tokenizer.tokenize(raw)
		stopped_tokens = []
		# remove stop words from tokens
		#stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]
		if lemmatized:
		  for t in tokens:
		    if t not in en_stop and len(t)>1:
		      pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([t])[0][1])
		      if pos:
		        stopped_tokens.append(lemmatizer.lemmatize(t,pos=pos))
		      else:
		        stopped_tokens.append(lemmatizer.lemmatize(t))
		  #     print(t,pos,nltk.pos_tag([t])[0][1])
		  # print(stopped_tokens)
		  #stopped_tokens = [lemmatizer.lemmatize(i,pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([i])[0][1])) for i in tokens if not i in en_stop and len(i)>1]
		else:
		  stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]


		# add tokens to list
		if len(stopped_tokens) >= len_th:
		  texts.append([stopped_tokens, i, id_list[c]]) # [pre-processed text (tokenized), original, id]
		  #original_docs.append([i,c])

		c += 1

	return texts

[nltk_data] Downloading package stopwords to /home/lem325/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lem325/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/lem325/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lem325/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
